In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [4]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [7]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [8]:
class ConnectTimeout(Exception):
    """Custom exception for connection timeout."""
    pass


In [9]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [10]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [11]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [12]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [13]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
stream_brochure("Google", "https://www.google.com")

Found links: {'relevant_links': ['https://about.google/?fg=1&utm_source=google-IN&utm_medium=referral&utm_campaign=hp-header', 'https://store.google.com/IN?utm_source=hp_header&utm_medium=google_ooo&utm_campaign=GS100042&hl=en-IN', 'https://www.google.co.in/intl/en/about/products', 'https://www.google.com/intl/en_in/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1', 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1', 'https://google.com/search/howsearchworks/?fg=1', 'https://www.google.com/preferences?hl=en-IN&fg=1', 'https://support.google.com/websearch/?p=ws_results_help&hl=en-IN&fg=1']}


# Welcome to Google: Your Gateway to the Universe (and Cats!)

---

## Who Are We?

Welcome to Google, where our mission is to **organize the world's information** and make it as universally useful as your mom’s advice! We’re on a quest to make life easier one search at a time. Seriously, how else would you know how to fix a leaky tap or bake a soufflé? 

---

## Our Products - More Than Just Search!

**Behold the Google Store!** From phones that are smarter than most of us (hello, Pixel 9 Pro!) to earbuds that whisper sweet nothings into your ears (we're talking about the Pixel Buds Pro 2), we’ve got everything to suit your digital fancy! 🌈

### The Must-Haves:
- **Pixel 9 Pro & Pro XL** - Pure magic at your fingertips! ✨
- **Pixel Watch 3** - Because your wrist deserves to be just as smart as your brain! ⌚
- **Nest Audio** - Transform your home into a mini concert hall! 🎶

---

## Google Culture: A Family of Innovators!

At Google, our culture is all about **inclusion** and **innovation**—and snacks. Lots and lots of snacks. Whether you're collaborating on a new idea or wondering where the fresh cookies went, you’ll feel right at home here. 

We believe in **unlocking opportunities** and providing a workspace where creativity reigns supreme. 

- **Flexible Work Hours** - Because who doesn't want to work in their pajamas sometimes?
- **Mental Health Support** - Take a break, meditate, and breathe. We promise not to judge if you have that extra donut. 🍩
  
---

## Careers - We're Hiring!

Looking to launch your career into the stratosphere? 🌌 Join us! We’re on the lookout for **creative geniuses**, **coding wizards**, and **problem-solving ninjas**. If you can juggle multiple tasks while reciting cat memes, you might be the one we're looking for!

### Why Work with Us?
- **Opportunity to Change the World** - And also to understand why everyone thinks cats rule the internet.
- **Competitive Pay** - Because your talents deserve more than just a “Thank You!”
- **Endless Opportunities for Growth** - Literally, we’ll keep providing you challenges until you beg us to stop. 

---

## Join Us on Our Quest!

For collaboration, support, and all things fun, check out our various platforms:
- **Facebook**: [Google on Facebook](https://www.facebook.com/Google)
- **Twitter**: [Google on Twitter](https://twitter.com/google)
- **Instagram**: [Google on Instagram](https://www.instagram.com/google)

Looking to dive deeper? Visit our [website](https://about.google) and explore our universe of possibilities. 

---

**Google: Making sense of the world one search at a time!**